In [ ]:
from datetime import datetime, date, timedelta, time
import pandas as pd
#import numpy as np
#import re
#import logging
#import json
#import sqlite3
#import yaml
#import itertools
#import os
#import pytz
#import hashlib
#from shutil import copyfile

In [ ]:
url='url'
filepath='C:/Users/Neon/Desktop/GitHub/download_entsoe/2017_2_AggregatedGenerationPerType.csv'


In [ ]:
def read_entsoe_transparency(filepath, url):

    headers = ['region', 'variable', 'attribute', 
                   'source', 'subsource','region_type','web']

    df = pd.read_csv(
        filepath,
        sep='\t',
        encoding='utf-16',
        header=0,
        index_col=['DateTime'],
        parse_dates=True,
        converters={'MapCode': lambda y: y[:2]},
        nrows=500,
        )

    df['AreaName']=[x.replace(' CA','').replace(' BZ','') 
                    for x in df['AreaName']]

    df.drop(['year', 'month', 'day'], 
            axis=1, inplace=True)
    
    df.rename(columns ={
        'ActualConsumption': 'consumption', 
        'ActualGenerationOutput': 'generation'},inplace=True)
    
    df.set_index(
        ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
        append=True, inplace=True)
    
    df=df.unstack('MapCode')
    df=df.unstack('ProductionType_Name')
    df=df.unstack('AreaName')
    df=df.unstack('AreaTypeCode')
    
    df=pd.concat(
        [df], 
        keys=['ENTSO-E'], 
        names=['source', 'attribute', 'region', 'variable', 'subsource','region_type'],
        axis=1)
    
    df=pd.concat([df], keys= url, names=['web'], axis=1)
    df=df.reorder_levels(headers, axis=1)
    
    return df